In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import scipy.optimize as spo
import scipy.signal as sps
from pathlib import Path
import re

from spectrometer import FID1D

In [ ]:
# Define ETH colors
eth_colors = [
    "#215CAF",  # ETH blue
    "#B7352D",  # ETH red
    "#627313",  # ETH green
    "#A7117A",  # ETH purple
    "#8E6713",  # ETH bronze
    "#007894",  # ETH petrol
    "#6F6F6F",  # ETH gray
]

In [ ]:
# Load raw *.fid data
fids: list[FID1D] = []
for file in Path("../data/some_folder/").glob("*.fid"):
    fids.append(FID1D.from_file(file))

# Extract pulse lengths from metadata
pattern = r"length=(.*)us,delay="
pulse_lengths_us = [float(re.search(pattern, str(fid.pulse))) for fid in fids]

In [ ]:
# Process
signal_strengths = []
for i, fid in enumerate(fids):
    x, y = fid.simple_fft(phase_shift_kwargs=False)
    y2 = np.abs(y) ** 2
    integral = np.trapz(x=x, y=y2)
    signal_strengths.append(integral)

In [ ]:
# Sort data by pulse lengths
pulse_lengths_us, signal_strengths = zip(
    *sorted(zip(pulse_lengths_us, signal_strengths))
)

# Plot raw data
cm = 1 / 2.54
fig, axes = plt.subplots(
    figsize=(16.0 * cm, 8.0 * cm),  # Wide Layout: ~16cm, Margin Layout: 10.7cm
    layout="constrained",
)
axes.plot(
    pulse_lengths_us,
    signal_strengths,
    color=eth_colors[0],
    linestyle="",
    marker="o",
    markersize=0.8,
)
axes.set_ylabel("Signal strength [a.u.]")
axes.set_xlabel("Pulse length [μs]")

# Adjust axis
axes.spines[["top", "right"]].set_visible(False)
axes.spines[["left", "bottom"]].set_position(("outward", 20))
plt.rcParams["axes.autolimit_mode"] = "round_numbers"
plt.rcParams["axes.xmargin"] = 0
plt.rcParams["axes.ymargin"] = 0
plt.setp(
    [
        *axes.lines,
        *axes.tables,
        *axes.artists,
        *axes.images,
        *axes.patches,
        *axes.texts,
        *axes.collections,
    ],
    clip_on=False,
)

# Set axis colour to ETH Grey
color = "#575757"  # normal grey: "#6F6F6F", dark grey: "#575757"
axes.tick_params(colors=color)
axes.spines[["left", "bottom", "top", "right"]].set_color(color)
axes.xaxis.label.set_color(color)
axes.yaxis.label.set_color(color)

# Change font
plt.setp(
    [
        axes.title,
        axes.xaxis.label,
        axes.yaxis.label,
        *axes.get_xticklabels(),
        *axes.get_yticklabels(),
        *(axes.get_legend().get_texts() if axes.get_legend() else ()),
    ],
    family=["Tex Gyre Pagella"],
)

# Adjust ticks
axes.tick_params(axis="both", direction="in")

fig.savefig("rabi_nutation_raw.pdf")

In [ ]:
def fit_decaying_squared_sinusoid(x: npt.NDArray, y: npt.NDArray) -> dict:
    """Fit a decaying squared sine wave to the input sequence

    f(x) = a * e^-lt * sin^2 (w * t + p) + c

    Returns:
        Fitting parameters "amp", "omega", "phase", "offset", "freq", "period" and "fitfunc"
    """
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)

    def decaying_sinusoid_squared(t, amplitude, lambda_, freq, phase, offset):
        return (
            amplitude * np.exp(-lambda_ * t) * np.sin(2 * np.pi * freq * t + phase) ** 2
            + offset
        )

    # Guess initial fitting parameters
    frequencies = np.fft.fftfreq(len(x), (x[1] - x[0]))  # assume uniform spacing
    fft = abs(np.fft.fft(y))
    # excluding the zero frequency "peak", which is related to offset
    guess_frequency = np.abs(frequencies[np.argmax(fft[1:]) + 1])
    guess_amplitude = np.std(y) * 2.0**0.5
    guess_offset = np.mean(y)
    guess_phase = 0
    guess_lambda = 0
    guess = (guess_amplitude, guess_lambda, guess_frequency, guess_phase, guess_offset)

    popt, _pcov = spo.curve_fit(decaying_sinusoid_squared, x, y, p0=guess)
    return {
        "amplitude": popt[0],
        "lambda": popt[1],
        "frequency": popt[2],
        "phase": popt[3],
        "offset": popt[4],
        "function": lambda t: decaying_sinusoid_squared(t, *popt),
    }

In [ ]:
# Try to plot simple least squares fit
fit = fit_decaying_squared_sinusoid(pulse_lengths_us, signal_strengths)
pulse_lengths_us_fine = np.linspace(
    pulse_lengths_us[0], pulse_lengths_us[-1], len(pulse_lengths_us) * 10
)
axes.plot(
    pulse_lengths_us_fine,
    fit["function"](pulse_lengths_us_fine),
    linestyle="-",
    color=eth_colors[1],
    linewidth=2,
)

# Save plot
fig.savefig("rabi_nutation_fit.pdf")
fig